# Statistics of Trends and Permutation Cycles

## Trends and Cycles

The statistics of permutation cycles is well-studied.
If we could show a relationship between permutation cycles and trends, we could use the statistics of permutation cycles to generate statistics of trends.
Let's try.

### Cycles to trends

A standard way to print permutation cycles is as lists of lists.  You can write the permutation $[0, 1,2,3,4,5] -> [1,3,0,4,2]$
as the cycle $[[0,1,3],[2,4]]$, (that's "0 becomes 1, 1 becomes 3, 3 becomes 0, and so on),
and the identity permutation $[0,1,2,3,4] -> [0,1,2,3,4]$ is $[[0],[1],[2],[3],[4]]$ (each element maps to itself).

It's also conventional to write each cycle with its smallest number first, and sort the cycles by their first numbers, so although the two are equivalent, you'd write
$[[4,2],[3,0,1]]$ as $[[0,1,3],[2,4]]$

How shall we turn permutation cycles into trendlists, and vice-versa?

Start by generating a list of $N$ random floats. We could generate random floats with `trendlist.rands()`, but, to ensure that we all see the same example, let's just hand-craft a tiny list.

In [ ]:
rnds = [0.43, 0.01, 0.95]
n = len(rnds)

Next, generate all permutations of the indices.

In [ ]:
from itertools import permutations
for permutation in permutations(range(n)):
    print(permutation)

Decompose those into cycles. We can use the Python package for symbolic computation, `sympy`.
If you need to install it, just `pip install sympy`.

In [ ]:
from sympy.combinatorics import Permutation as perm
for permutation in permutations(range(n)):
    print(perm(permutation).full_cyclic_form)

Turn these indices back into the floats they represent.

In [ ]:
def to_floats(cycle): # turn a cycle into corresponding floats
    return [rnds[elem] for elem in cycle]
def cycles_to_floats(cycles): # turn a permutation in cycle-list notation back into floats
    return [to_floats(cycle) for cycle in cycles]
for permutation in permutations(range(n)): # map each cycle list to a list of lists of floats
    float_cycles = cycles_to_floats(perm(permutation).full_cyclic_form)
    print(float_cycles)

Every list of random floats has exactly one circular permutation that's a trend.
Rotate each cycle to that trend.

In [ ]:
from trendlist.simple import is_trend
def rotate_to_trend(floats):  # rotate a list of floats to a single trend
    while not is_trend(floats):
        floats = floats[1:] + floats[:1]  # rotate the list one element
    return floats
def to_trends(float_cycles): # rotate each list in the list of lists to its trend order
    return [rotate_to_trend(float_cycle) for float_cycle in float_cycles]
for permutation in permutations(range(n)):
    float_cycles = cycles_to_floats(perm(permutation).full_cyclic_form)
    print(to_trends(float_cycles))


Finally, we know that in a trendlist, the means decrease monotonically.

In [ ]:
from statistics import mean

def to_trendlist(permutation):  # map a permutation to a trendlist
    float_cycles = cycles_to_floats(perm(permutation).full_cyclic_form)
    return sorted(to_trends(float_cycles), key=mean, reverse=True)
    
for permutation in permutations(range(n)):
    print(f"{permutation} -> {to_trendlist(permutation)}")

To review, for a long list of random floats,

* replace the floats by their indices
* treat the list of integers as a permutation and decompose it into cycles
* turn the indices back into floats
* rotate each cycle into a trend
* sort the trends by their means, in decreasing order

Collect all these into a single functionand then maps permutations on $S_n$ into trendlists, here:

This maps the list of all permutations on ${1..n}, S_n$, uniquely into trendlists. The steps are easy to run in reverse, and you can give it a try, here:

## Statistics of Permutations

### "It is better to look good than to feel good." – Billy Crystal as Fernando Lamas

We'll begin by creating a couple of utility methods for printing text.
Here's how to print text in bold and in normal.

In [ ]:
print('\x1b[1m' + "hello" + '\x1b[0m' + "world")

Here is a routine that encapsulates that.

In [ ]:
def c_print(s, reverse = False):
    ansi = "\x1b[1m" if reverse else "\x1b[0m"
    print(f"{ansi}{s}", end = "")
c_print("hello to ")
c_print("all ", reverse=True)
c_print("the world")

Next, let's print every *other* cycle in a list of cycles in bold.

In [ ]:
def print_cycles(cycles):
    for n, cycle in enumerate(cycles):
        c_print(cycle, reverse = n%2)
    c_print("\n")  # reset to normal

print_cycles([[1, 2], [3,4], [5, 6], [7, 8]])
print("hello")

This will prove useful shortly, but first let's go back to permutations.

## Generating All Permutations

First, let's generate all permutations of a set with the standard library module $itertools$.

In [ ]:
from itertools import permutations
for permutation in permutations({1, 2, 3}):  # all permutations of {1, 2, 3}
    print(permutation)

Next, we'll use *sympy.combinatorics* to decompose a permutation into cycles.

In [ ]:
from sympy.combinatorics import Permutation as perm # break a single permutation into cycles
perm((0, 1, 3, 2)).full_cyclic_form

This pair lets us decompose all permutions of a set into cycles.

In [ ]:
for permutation in permutations(range(3)):  # put those two together
    print(perm(permutation).full_cyclic_form)

If we just collect all those cycles into one, giant list, we can ask questions about every cycle in all permutations.

In [ ]:
# next, collect all cycles, over all permutations

all_cycles = []
for permutation in permutations(range(3)):
    all_cycles.extend(perm(permutation).full_cyclic_form)
all_cycles

That's a bit hard-to-read, so let's group them by cycle length.

In [ ]:
cycles_by_length = {}  # next, group those by length: all length-1 cycles, all length-2, etc.
for length in (range(1, 4)):
    cycles_by_length[length] = sorted([cycle for cycle in all_cycles if len(cycle) == length])
cycles_by_length

Finally, let's squeeze out all the air in each line, removing the commas, spaces, and square and curly brackets. We can separate adjacent trends by printing them in different weights by using the *print_cycles()* routine we defined earlier.

In [ ]:
# now pretty-print them by length, with all the meta-characters squeezed out
# switch boldness to highlight each new cycle
for length, list in cycles_by_length.items():
    print_cycles(["".join(map(str, cycle)) for cycle in list])

That was at least easy, but let's collect all the routines above into functions
and see what if it might tell us anything.

In [ ]:
# putting it all together
def all_cycles(n):
    all_cycles = []
    for permutation in permutations(range(n)):
        all_cycles.extend(perm(permutation).full_cyclic_form)
    return all_cycles

def cycles_by_length(n):
    cycles = all_cycles(n)
    cycles_by_length = {}
    for length in (range(1, n+1)):
        cycles_by_length[length] = sorted([cycle for cycle in cycles if len(cycle) == length])
    return cycles_by_length

def cycle_block(n):
    cycle_block = []
    cbl = cycles_by_length(n)
    for length, list in cbl.items():
        cycle_block.append(["".join(map(str, cycle)) for cycle in list])
    return cycle_block

for cycle_length in cycle_block(4):
    print_cycles(cycle_length)

Oh! For both $n = 3$ and $n = 4$, the rows are all the same length.
Is that true for smaller $n$?

In [ ]:
for cycle_length in cycle_block(2):
    print_cycles(cycle_length)

(These look similar, but the first line is two cycles, each length one, which fix the two elements,
while the second is a single cycle of length two, which exchanges the two elements.)

Longer $n$?  The lines are so long they won't fit cleanly onto all screens, but you can still see the rows line up: they're the same length.

In [ ]:
for cycle_length in cycle_block(5):
    print_cycles(cycle_length)

Here's a conjecture:

In a cycle block made from all permutations of a sequence of length N,

1. There are $N$ lines.
1. There are $N*N!$ elements in all the cycles.
1. Each line is the same length: N!

The first isn't surprising. For example, for sequences of length $8$, the permutation $(0, 1, 2, 3, 4, 5, 6, 7)$ has cycles $[[0],[1],[2],...[7]]$ -- all length $1$ -- while $(1, 2, 3, 4, 5, 6, 7, 0)$ has the single, $8$-long cycle $[[0, 1, 2, 3, 4, 5, 6, 7]]$.

It's trivial to construct permutations that have any cycle length in between. Try it yourself.

The second of these also makes sense. The total number of elements in all the cycles 
is the total number of elements in all permutations. 
For sequences of length 8, there are 8! permutations, and each permutation has 8 elements:
$(0, 1, ..., 6, 7), (0, 1, ..., 7, 6), ...$
A sequence of length $N$ has $N!$ permutations, each one with $N$ elements

The most remarkable is the third. Let's defer proving that for a moment
and look at some consequences.

1. There are $n!/k$ cycles of length $k$.
2. On average, there are $1/k$ cycles of length $k$ in each permutation.

For example, on average, every permutation has one cycle of length 1.

3. The total number of cycles in all permutations
is $\sum_1^n{n!/k}$ = n!$\sum_1^n{1/k} = n!H_n$,
where $H_n$ is the n-th harmonic number: $1 + 1/2 + 1/3 + ... + 1/N$
4. The average number of cycles in the $n!$ permutations is $n!H_n/n! = H_n$.
5. The average number of cycles is about the log of the sequence length, because $lim_{n->\infty}{(H_n-ln(n))} = \gamma$,
where $\gamma = 0.57721...$ is the Euler-Mascharoni constant.

This isn't shocking. You probably remember that $\int_0^n{(1/x)}dx = ln(n)$ 

6. The average length of a cycle is $n/H_n \approx n/ln(n)$
7. The expected number of cycles longer than $k$ is 
$\approx (ln(n) - ln(k)) = ln(n/k)$


Picturing the easy-to-visualize cycle block in your head, leads to each of these points right away.

For 1-6 above, if you substitute *trend* for *cycle*, the statements should still hold.

## "What Would You Pay? But Wait ... There's More!" – Ronald Popile

Another number to know, to help build up a better picture of typical trend decompositions, is the average *longest* trend.

Again, just looking it up, we learn that the average longest permutation is nearly two-thirds the length of the sequence. The actual expected length of the longest trend is $\lambda*N$, where N is the length of the sequence
and $\lambda = 0.62432998854355087099293638310083724\dots$ is the *Golomb-Dickman* constant.

Like $\pi$, $e$, $\gamma$, and other odd, named constants, the Golomb-Dickman constant appears hither and yon, without warning, in all manner of formulae. You can read more about it in Wikipedia.

A typical sequence of length 25,000 long has around ten trends, with a single, long trend, near the middle, making up about 2/3 of the sequence. It has about 16,000 elements, flanked by 4 or 5 small trends on either side, less than 1,000 long apiece.

I find it's easier to keep this picture in my head by thinking about this as the days of an average, 70-year life.

A completely random, 70-year lifespan would have, on average, four or five, two-to-four year trends up until the start of puberty, followed by quite a long stretch, in which things are getting better, on average, every day, until about age 55. In the remaining 15 years, you'd expect to see another four or five, two-to-four year trends.

You can find a lot more about the statistics of random permutations -- identical to the statistics of trends -- on the valves and tubes of the interwebs.

## Every Line in the Cycle Block Is the Same Length

Let's now go back and show that each line is the same length.
There are lots of ways to skin this cat. We'll try one that just uses counting.

Start with an example.
How many cycles are $(0)$? That is, how many permutations out of $n!$ map $0$ to itself?

In math-speak, this is called "fixing 0." This may make you nervous already, since that's a use of the word "fix" akin to the one in the sentence, "I just got my cat fixed," where "fixed" really means "broken." Stick with us. Keep telling yourself, like Sir Galahad, ["My strength is as the strength of ten because my heart is pure."](https://en.wikisource.org/wiki/Page:Poems_that_every_child_should_know_(ed._Burt,_1904).djvu/291)


In permutations that fix $0$, the rest of the numbers, ${1,..,n-1}$ can map to anything, including themselves, so there are exactly $(n-1)!$ instances of $(0)$ in our collection.

Of course, $0$ isn't special, so there are also $(n-1)!$ instances of $(1)$, and every other digit, right up to $(n-1)!$ instances of $(n)$.  Some of these will occur in the same permutations, but we don't care, we're just counting the number that fix $k$, for each $0 <= k <= n-1$

In other words, there are exactly $n*(n-1)! = n!$ cycles of length $1$ The first row of our cycle block has length $n!$

How about the second row?
Let's count every permutation that contains the cycle $(0, 1)$ –
every permutation that happens to swap $0$ with $1$.

If we ignore those two for a second, there are $(n-2)!$ ways to permute the remaining $(n-2)$ digits, so there are $(n-2)!$ instances of the cycle $(0, 1)$.

Again, $(0, 1)$ isn't special, so there are also $(n-2)!$ instances of $(0, 3), (0, 4), ...(n-1, n)$  Since there are $n \choose 2$ different cycles of length $2$, there are ${n \choose 2}*(n-2)! = (n*(n-1)/2)*(n-2)! = n!/2$ cycles of length 2.  The second row in our cycle block also has length $n!$


The $k$th row has cycles of length $k$. The first of these is $(0, .. k-1)$. 
Reasoning the same way, there are exactly $(n-k)!$ of these.

Here comes a tiny twist:

If you think about all permutations of ${0, 1, 2, 3}$,
we've been talking about the cycle $(0, 1, 2)$. There are $(n-2)! = 2! = 2$ of these.
Before moving on to 3-cycles containing other numbers, 
let's pause, and count all instances of $(0, 2, 1)$, which is a different cycle!
There are 2 of these, too.


In general, there are $n \choose k$ ways to choose numbers for a $k$-cycle, 
but $(k-1)!$ different ways to arrange those numbers into a $k$-cycle.
And each of those will occur $(n-k)!$ times.


So, how long is the $k$th row? (Number of ways to pick a set of $k$)$*$(Number of cycles you can make with those $k$)$*$(Number of permutations of the $(n-k)$ you didn't pick)$*$(Length of the cycle) = ${n \choose k} * (k-1)! * (n-k)! * k = (n!/k!(n-k)!) * k! * (n-k)! = n!$

Voila!  Or, as Euclid used to say, *"ὅπερ ἔδει δεῖξαι"*

If your tastes run to proofs with functors, bivariate generating functions, and partial derivatives, you can find one at [The n-Category Café.](https://golem.ph.utexas.edu/category/2019/11/random_permutations_part_5.html)